<html>
<body>
<h1 style = "color:blue"> Classify 10, 20 & 50 cent coins into two different categories</h1>

<h2 style = "color:red"> Abstract: </h2>
<p>The goal of this notebook is to build a convNN clasification using Tensor Flow library that is desinged to classify a given image of an euro coin into three different categories:.</p>
<p> a) Categorie 0 = 10 cents. </p>
<p> b) Categorie 1 = 20 cents. </p>
<p> c) Categorie 2 = 50 cents.

<h2 style = "color:red"> Dataset: </h2>
<p>The dataset employed in the training process is given by the Euro Coin dataset.</p>
</body>
</html>


In [ ]:
import os, cv2, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from coins_utils import *

TRAIN_DIR = 'CandidateDataSet2/augmented_and_resized_brightness_yuv_RAW/50centVs20centVs10cent/'
#TRAIN_DIR = 'CandidateDataSet2/augmented_and_resized_RAW/50centVs20centVs10cent/'

images_full = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if '.jpg' in i]
random.shuffle(images_full)
n_images = len(images_full)
print("Number of images in full dataset "+str(n_images))
print("------------------------------------")


#Splitting of the input dataset into train, validation and test set.
train_to_full = 0.3 # Fraction of images used for training
validation_to_full = 0.8 # Fraction of images used for validation
train_to_full_index = int(n_images*train_to_full)
validation_to_full_index = int(n_images*validation_to_full)

#train_images = sorted(images_full[:train_to_full_index])
#validation_images =  sorted(images_full[train_to_full_index:validation_to_full_index])
#test_images =  sorted(images_full[validation_to_full_index:])

train_images = images_full[:train_to_full_index]
validation_images =  images_full[train_to_full_index:validation_to_full_index]
test_images =  images_full[validation_to_full_index:]

n_train_images = len(train_images)
n_validation_images = len(validation_images)
n_test_images = len(test_images)

print("Total number of train images "+str(n_train_images))
print("Total number of validation images "+str(n_validation_images))
print("Total number of test images "+str(n_test_images))
if n_train_images+n_validation_images+n_test_images - n_images != 0:
    print("Warning, there is a problem in the splitting of images")

train = prep_data(train_images, colorSpace = "YUV")
validation = prep_data(validation_images, colorSpace = "YUV")
test = prep_data(test_images, colorSpace = "YUV")

#To make them easier to operate in future
train_images_name = [i.split('/')[-1] for i in train_images]
validation_images_name = [i.split('/')[-1] for i in validation_images]
test_images_name = [i.split('/')[-1] for i in test_images]
print(len(train_images_name), len(validation_images_name), len(test_images_name))

# Getting the LABELS

In [ ]:
## read the labels.
labels = pd.read_csv(TRAIN_DIR+'/labels_50centVs20centVs10cent.csv')

labels_train_df = labels[labels["FileName"].isin(train_images_name)]
labels_validation_df = labels[labels["FileName"].isin(validation_images_name)]
labels_test_df = labels[labels["FileName"].isin(test_images_name)]


#Example get the CoinType field and create array with labels (This is not very efficient...)
#iloc is there because for some reason it won't give me values with 'CoinType'.
nb_classes = 3
labels_train_nocategorical = []
for kitem in train_images_name:
        labels_train_nocategorical.append(labels[labels["FileName"] == kitem].iloc[0][3])   
labels_train_nocategorical= np.array(labels_train_nocategorical)
    
labels_validation_nocategorical = []
for kitem in validation_images_name:
        labels_validation_nocategorical.append(labels[labels["FileName"] == kitem].iloc[0][3]) 
labels_validation_nocategorical= np.array(labels_validation_nocategorical)


labels_test_nocategorical = []
for kitem in test_images_name:
        labels_test_nocategorical.append(labels[labels["FileName"] == kitem].iloc[0][3])     
labels_test_nocategorical= np.array(labels_test_nocategorical)


#Convert to categorical labels (vectors)
from keras.utils import np_utils
labels_train = np_utils.to_categorical(labels_train_nocategorical, nb_classes)
labels_validation = np_utils.to_categorical(labels_validation_nocategorical, nb_classes)
labels_test = np_utils.to_categorical(labels_test_nocategorical, nb_classes)

# Sanity check, we are reading everything properly

In [ ]:
#Print some coins
def printSomeCoints(img, train_images_name, labels, ncoins):
    for i in range(0,ncoins):
        print(train_images_name[i])
        if labels[i] == 0.:
            print("This is 10 cent")
        if labels[i] == 1.:
            print("This is 20 cent")
        if labels[i] == 2.:
            print("This is 50 cent")
        plt.figure(figsize=(10,5))
        print(train[i].shape)
        plt.imshow(cv2.cvtColor(train[i], cv2.COLOR_YUV2RGB))
        plt.show()
    
    
printSomeCoints(train, train_images_name, labels_train_nocategorical, 30)

<html>
<body>
<h2 style = "color:red"> Building the classificator (work in progress..): </h2>
<p> Use the training dataset to build the classificator using a ConvNN.</p>
</body>
</html>

In [ ]:
#Loading keras deep learnig libraries to build the model: https://keras.io/ 
from keras.layers import Input, Dense, Flatten, Dropout, Activation, advanced_activations, Convolution2D, MaxPooling2D
from keras.models import Model, model_from_json
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import initializations

## Standard Architecture

In [ ]:
#Model hyperparameters
batch_size = 32 # in each iteration, we consider batch_size training examples at once
num_epochs = 15 # we iterate num_epochs times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 62 # ...switching to 64 after the first pooling layer
conv_depth_3 = 128 # ...switching to 64 after the first pooling layer
conv_depth_4 = 256 # ...switching to 64 after the first pooling layer
conv_depth_5 = 512 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with this probability 
drop_prob_2 = 0.5 # dropout in the FC layer with this probability 
hidden_size = 64 # the FC layer will this neurons
data_augmentation = False # Whether to use or not data augmentation

ROWS = 100
COLS = 100
CHANNELS = 3

NN = False
ConvNN = True
if ConvNN == True:
    #Architecture
    inp = Input(shape=(ROWS, COLS, CHANNELS)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
    conv_2 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_1)
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    conv_3 = Convolution2D(conv_depth_3, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_2)
    #pool_3 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    #conv_4 = Convolution2D(conv_depth_4, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_3)
    #pool_4 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    #conv_5 = Convolution2D(conv_depth_5, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_4)
                       

    flat = Flatten()(conv_3)
    hidden = Dense(hidden_size, activation='tanh')(flat)
    out = Dense(nb_classes, activation='sigmoid')(hidden)
    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

if NN == True:
    #Architecture
    inp = Input(shape=(ROWS, COLS, CHANNELS)) # N.B. depth goes first in Keras!                  
    flat = Flatten()(inp)
    hidden = Dense(hidden_size, activation='tanh')(flat)
    out = Dense(nb_classes, activation='sigmoid')(hidden)
    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

    
#print the summary of the architecture
model.summary()

#Visulize the model if desired
#from keras.utils.visualize_util import plot
#plot(model, to_file='Example_of_CNN_CatsVsDogs.pdf')

# reduce the learning rate by factor of 0.5 if the validation loss does not get lower in 7 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0000001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')


# Let's train the model using SGD
#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

# Let's train the SGD model WITHOUT using data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(train, labels_train, batch_size=batch_size, nb_epoch=num_epochs, validation_data=(validation, labels_validation), shuffle=True, callbacks=[reduce_lr,early_stopping])
    


## Reduced (And deeper) architecture

In [ ]:
#Model hyperparameters
batch_size = 32 # in each iteration, we consider batch_size training examples at once
num_epochs = 100 # we iterate num_epochs times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 8
conv_depth_2 = 16
conv_depth_3 = 32
conv_depth_4 = 64
conv_depth_5 = 64
conv_depth_6 = 64
conv_depth_7 = 128
conv_depth_8 = 128
#conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
#conv_depth_2 = 62 # ...switching to 64 after the first pooling layer
#conv_depth_3 = 128 # ...switching to 64 after the first pooling layer
#conv_depth_4 = 256 # ...switching to 64 after the first pooling layer
#conv_depth_5 = 512 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with this probability 
drop_prob_2 = 0.5 # dropout in the FC layer with this probability 
hidden_size = 24 # the FC layer will this neurons
data_augmentation = False # Whether to use or not data augmentation

ROWS = 100
COLS = 100
CHANNELS = 3

NN = False
ConvNN = True
if ConvNN == True:
    #Architecture
    inp = Input(shape=(ROWS, COLS, CHANNELS)) # N.B. depth goes first in Keras!
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
    conv_2 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_1)
    pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    pool_2_d = Dropout(drop_prob_1)(pool_2)
    conv_3 = Convolution2D(conv_depth_3, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_2_d)
    pool_3 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
    pool_3_d = Dropout(drop_prob_1)(pool_3)
    conv_4 = Convolution2D(conv_depth_4, kernel_size, kernel_size, border_mode='same', activation='tanh')(pool_3_d)
    #pool_4 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
    conv_5 = Convolution2D(conv_depth_5, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_4)
    #pool_5 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_5)
    conv_6 = Convolution2D(conv_depth_6, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_5)
    #pool_6 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_6)
    conv_7 = Convolution2D(conv_depth_7, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_6)
    #pool_7 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_7)
    #pool_7_d = Dropout(drop_prob_1)(pool_7)
    conv_8 = Convolution2D(conv_depth_8, kernel_size, kernel_size, border_mode='same', activation='tanh')(conv_7)
                       

    flat = Flatten()(conv_8)
    hidden = Dense(hidden_size, activation='tanh')(flat)
    hidden_d = Dropout(drop_prob_2)(hidden)
    out = Dense(nb_classes, activation='sigmoid')(hidden_d)
    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

if NN == True:
    #Architecture
    inp = Input(shape=(ROWS, COLS, CHANNELS)) # N.B. depth goes first in Keras!                  
    flat = Flatten()(inp)
    hidden = Dense(hidden_size, activation='tanh')(flat)
    out = Dense(nb_classes, activation='sigmoid')(hidden)
    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

    
#print the summary of the architecture
model.summary()

#Visulize the model if desired
#from keras.utils.visualize_util import plot
#plot(model, to_file='Example_of_CNN_CatsVsDogs.pdf')

# reduce the learning rate by factor of 0.5 if the validation loss does not get lower in 7 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0000001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')


# Let's train the model using SGD
#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

# Let's train the SGD model WITHOUT using data augmentation
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(train, labels_train, batch_size=batch_size, nb_epoch=num_epochs, validation_data=(validation, labels_validation), shuffle=True, callbacks=[reduce_lr,early_stopping])
                       
                       

# Learning curves (check the performance)

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig('Accuracy_Example_Coin_Classifier_GoldenCoins.pdf')
plt.close()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig('Loss_Example_Coin_Classifier_GoldenCoins.pdf')
plt.close()

# Save 

In [ ]:
#SaveModel(model, "Model_Example_Coin_Classifier_GoldenCoins_Standard_yuv") #For the standard architecture
#SaveModel(model, "Model_Example_Coin_Classifier_GoldenCoins_Reduced_yuv") #For the reduced architecture
SaveModel(model, "Model_Example_Coin_Classifier_GoldenCoins_Deep_30p_dropout025_yuv") #For the reduced deeper architecture

# Load and Test model

In [ ]:
#Useful to test.
LoadAndTest = True
nTest = 45
if LoadAndTest == True:
    #loaded_model=LoadModel("Model_Example_Coin_Classifier_GoldenCoins_Standard_yuv") #For the standard architecture
    #loaded_model=LoadModel("Model_Example_Coin_Classifier_GoldenCoins_Reduced_yuv") #For the reduced architecture
    loaded_model=LoadModel("Model_Example_Coin_Classifier_GoldenCoins_Deep_30p_dropout025_yuv") #For the reduced deeper architecture
    # evaluate loaded model on test data
    loaded_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
    
    #score_train = loaded_model.evaluate(train, labels_train, verbose=0)
    #score_validation = loaded_model.evaluate(validation, labels_validation, verbose=0)
    #print "Training %s: %.2f%%" % (loaded_model.metrics_names[1], score_train[1]*100)
    #print "Validation %s: %.2f%%" % (loaded_model.metrics_names[1], score_validation[1]*100)    
    
    print("PREDICTIONS \n")
    reduced_test = test[:nTest]
    predictedLabel = loaded_model.predict(reduced_test)

    for i in range(0,nTest):
        print(" %.8f of 10 cents \n" % predictedLabel[i,0])
        print(" %.8f of 20 cents \n" % predictedLabel[i,1])
        print(" %.8f of 50 cents \n" % predictedLabel[i,2])

        plt.imshow(cv2.cvtColor(reduced_test[i], cv2.COLOR_YUV2RGB))
        plt.show()
    
        
    

## Testing with mixed coins images

In [ ]:
#Useful to test.
TEST_DIR = 'CandidateDataSet2/Processed_RAW/mix/'
#input_file = glob.glob(TEST_DIR+"*.jpg") #Try new Samples
input_file = [TEST_DIR+"P70216-160609_Candidate_6.jpg"]  #For debugging
print("--- 10 cents ---")
input_file.append(TEST_DIR+"P70216-160642_Candidate_10.jpg")
input_file.append(TEST_DIR+"P70216-160642_Candidate_20.jpg")
input_file.append(TEST_DIR+"P70216-160642_Candidate_21.jpg")
input_file.append(TEST_DIR+"P70216-160652_Candidate_27.jpg")
input_file.append(TEST_DIR+"P70216-160652_Candidate_25.jpg")
input_file.append(TEST_DIR+"P70216-160652_Candidate_26.jpg")
input_file.append(TEST_DIR+"P70216-160813_Candidate_37.jpg")
input_file.append(TEST_DIR+"P70216-161849_Candidate_4.jpg")
input_file.append(TEST_DIR+"P70216-161849_Candidate_18.jpg")
input_file.append(TEST_DIR+"P70216-161000_Candidate_34.jpg")
input_file.append(TEST_DIR+"P70216-160854_Candidate_38.jpg")
print("--- 10 cents : VALIDATION SET ---")
VAL_DIR = 'CandidateDataSet2/Processed_RAW/10cent/'
input_file.append(VAL_DIR+"P70216-134319_Candidate_14.jpg")
input_file.append(VAL_DIR+"P70216-134504_Candidate_21.jpg")
input_file.append(VAL_DIR+"P70216-134748_Candidate_5.jpg")
input_file.append(VAL_DIR+"P70216-134440_Candidate_9.jpg")
input_file.append(VAL_DIR+"P70216-134748_Candidate_9.jpg")
print("--- 20 cents ---")
input_file.append(TEST_DIR+"P70216-160459_Candidate_5.jpg")
input_file.append(TEST_DIR+"P70216-160524_Candidate_2.jpg")
input_file.append(TEST_DIR+"P70216-160630_Candidate_9.jpg")
input_file.append(TEST_DIR+"P70216-160630_Candidate_17.jpg")
input_file.append(TEST_DIR+"P70216-160652_Candidate_7.jpg")
input_file.append(TEST_DIR+"P70216-160813_Candidate_8.jpg")
input_file.append(TEST_DIR+"P70216-160854_Candidate_7.jpg")
input_file.append(TEST_DIR+"P70216-160854_Candidate_29.jpg")
input_file.append(TEST_DIR+"P70216-160652_Candidate_7.jpg")
print("--- 20 cents : VALIDATION SET ---")
VAL_DIR = 'CandidateDataSet2/Processed_RAW/20cent/'
input_file.append(VAL_DIR+"P70210-172818_Candidate_14.jpg")
input_file.append(VAL_DIR+"P70210-172835_Candidate_1.jpg")
input_file.append(VAL_DIR+"P70210-173008_Candidate_3.jpg")
input_file.append(VAL_DIR+"P70210-173043_Candidate_15.jpg")
input_file.append(VAL_DIR+"P70210-173134_Candidate_13.jpg")
print("--- 50 cents ---")
input_file.append(TEST_DIR+"P70216-160459_Candidate_6.jpg")
input_file.append(TEST_DIR+"P70216-160434_Candidate_5.jpg")
input_file.append(TEST_DIR+"P70216-160549_Candidate_5.jpg")
input_file.append(TEST_DIR+"P70216-160609_Candidate_6.jpg")
input_file.append(TEST_DIR+"P70216-160630_Candidate_13.jpg")
input_file.append(TEST_DIR+"P70216-160813_Candidate_24.jpg")
input_file.append(TEST_DIR+"P70216-161000_Candidate_26.jpg")
input_file.append(TEST_DIR+"P70216-161849_Candidate_13.jpg")
input_file.append(TEST_DIR+"P70216-161845_Candidate_10.jpg")
print("--- 50 cents : VALIDATION SET ---")
VAL_DIR = 'CandidateDataSet2/Processed_RAW/50cent/'
input_file.append(VAL_DIR+"P70210-165438_Candidate_20.jpg")
input_file.append(VAL_DIR+"P70210-165500_Candidate_7.jpg")
input_file.append(VAL_DIR+"P70210-165718_Candidate_4.jpg")
input_file.append(VAL_DIR+"P70210-165812_Candidate_3.jpg")
input_file.append(VAL_DIR+"P70210-165858_Candidate_4.jpg")
LoadAndTest = True
if LoadAndTest == True:
    #loaded_model=LoadModel("Model_Example_Coin_Classifier_GoldenCoins_Standard_yuv") #For the standard architecture
    loaded_model=LoadModel("Model_Example_Coin_Classifier_GoldenCoins_Deep_30p_dropout025_yuv") #For the reduced architecture
    # evaluate loaded model on test data
    #loaded_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
    print("PREDICTIONS \n")
    for kfile in input_file:
        im = Image.open(kfile)
        imarray = np.array(im)
        imgResizedTest = cv2.resize(imarray, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
        imgResizedTest = cv2.cvtColor(imgResizedTest, cv2.COLOR_RGB2YUV)
        imgResizedTest = [imgResizedTest for i in range(0,1)]
        imgResizedTest = np.array(imgResizedTest)        
        predictedLabel = loaded_model.predict(imgResizedTest)
        print(" %.8f of 10 cent \n" % predictedLabel[0,0])
        print(" %.8f of 20 cent \n" % predictedLabel[0,1])
        print(" %.8f of 50 cent \n" % predictedLabel[0,2])
        print(predictedLabel)
        
        #imgResizedTest = cv2.cvtColor(imgResizedTest[0], cv2.COLOR_YUV2BGR)
        #imgResizedTest = cv2.cvtColor(imgResizedTest, cv2.COLOR_BGR2RGB)
        plt.imshow(cv2.cvtColor(imgResizedTest[0], cv2.COLOR_YUV2RGB))
        plt.show()